In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import io
import PIL
import time
from toolkit.common.constants import *
from toolkit.lap.gps_importer import load_track_from_mat, load_track_from_raw
from toolkit.motor.motor import Motor

In [ ]:
emrax_228_mv = Motor('EMRAX 228 MV')
emrax_228_mv.Ld = 0.000076
emrax_228_mv.Lq = 0.000079
emrax_228_mv.Rs = 0.008 #0.0071
emrax_228_mv.max_phase_current = 340.0
emrax_228_mv.max_rate = 6500.0
emrax_228_mv.Fl = 0.0355
emrax_228_mv.poles = 10.0

emrax_228_hv = Motor('EMRAX 208 HV')
emrax_228_hv.Ld = 0.000177
emrax_228_hv.Lq = 0.000183
emrax_228_hv.Rs = 0.018
emrax_228_hv.max_phase_current = 240.0
emrax_228_hv.max_rate = 6500.0
emrax_228_hv.Fl = 0.0542
emrax_228_hv.poles = 10.0

cur_motor = emrax_228_mv

In [ ]:

def calc_current_limit(current, points):
    angles = np.linspace(0, 2*np.pi, points)
    i_d = current * np.cos(angles)
    i_q = current * np.sin(angles)
    return i_d, i_q

def create_rectangle(iq_lim, id_lim):
    # create all the points needed to make a rectangle with the limits of the motor controller
    i_q, i_d = np.zeros(5), np.zeros(5)
    i_q[[0, 1, 4]] = iq_lim
    i_q[[2, 3]] = -iq_lim
    i_d[[0, 3, 4]] = id_lim
    i_d[[1, 2]] = -id_lim
    return i_d, i_q

In [ ]:
#load matlab data from a test run
# raw_track = load_track_from_mat('./Data/logs/20230723-0910607.mat')
raw_track = load_track_from_mat('./Data/logs/20231022-0910602-3laps.mat')
raw_track.keys()

In [ ]:
# plot the force vs speed for all the settings
fig = go.Figure()

current_lims = 600
torque = 160.0
speed = 3000.0
voltage = 400.0
i_dmtpa, i_qmtpa = cur_motor.calc_mtpa(np.linspace(0, 2 * cur_motor.i_max, 1000))
fig.add_trace(go.Scatter(x=i_dmtpa, y=i_qmtpa, mode='lines', name='MTPA', line=dict(color='green', width=2)))
i_d_torque = np.linspace(-current_lims, current_lims, 1000)
i_q_torque = cur_motor.calc_iq(torque, i_d_torque)
fig.add_trace(go.Scatter(x=i_d_torque, y=i_q_torque, mode='lines', name='Requested Torque Curve', line=dict(color='blue', width=2)))
i_d_emf, i_q_emf = cur_motor.calc_emf_limit(speed, voltage, 1000)
fig.add_trace(go.Scatter(x=i_d_emf, y=i_q_emf, mode='lines', name='EMF Circle', line=dict(color='white', width=2)))
i_d_winding, i_q_winding = cur_motor.calc_current_limit(1000)
fig.add_trace(go.Scatter(x=i_d_winding, y=i_q_winding, mode='lines', name='Current Limit', line=dict(color='red', width=2)))
i_d_eeprom, i_q_eeprom = create_rectangle(350, 120)
fig.add_trace(go.Scatter(x=i_d_eeprom, y=i_q_eeprom, mode='lines', name='MCU EEPROM Current Limit', line=dict(color='orange', width=2)))
i_d_gate, i_q_gate = calc_current_limit(380, 1000)
fig.add_trace(go.Scatter(x=i_d_gate, y=i_q_gate, mode='lines', name='MCU Gate Current Limit', line=dict(color='purple', width=2)))

fig.update_layout(template="plotly_dark", title_text=f"Iq and Id Current Target", height=1000, width=1000, xaxis_title="i_d", yaxis_title="i_q", xaxis_range=[-current_lims, current_lims], yaxis_range=[-current_lims, current_lims])
fig.show()

In [ ]:
# make a pandas dataframe for the data
# print(raw_track['MCM_Motor_Speed']['Time'][0, 0][0])
data = pd.DataFrame(columns=['time', 'w', 'dc_voltage', 'dc_current', 'torque_command', 'torque_feedback', 'i_d', 'i_q', 'i_d_commanded', 'i_q_commanded', 'v_d_Voltage', 'v_q_Voltage', 'id_t', 'iq_t', 't_t', 'w_t', 'v_max', 'v_d_t', 'v_q_t', 'p_t'])

data['time'] = raw_track['MCM_Motor_Speed']['Time'][0, 0][0]
data['time'] = data['time'] - data['time'][0]
print(data['time'])
data['w'] = raw_track['MCM_Motor_Speed']['Value'][0, 0][0]
data['dc_voltage'] = raw_track['MCM_DC_Bus_Voltage']['Value'][0, 0][0]
data['dc_current'] = raw_track['MCM_DC_Bus_Current']['Value'][0, 0][0]
data['torque_command'] = raw_track['MCM_Commanded_Torque']['Value'][0, 0][0]
data['torque_feedback'] = raw_track['MCM_Torque_Feedback']['Value'][0, 0][0]
data['i_d'] = raw_track['MCM_Id']['Value'][0, 0][0]
data['i_q'] = raw_track['MCM_Iq']['Value'][0, 0][0]
data['i_d_commanded'] = raw_track['MCM_Id_Command']['Value'][0, 0][0]
data['i_q_commanded'] = raw_track['MCM_Iq_Command']['Value'][0, 0][0]
data['v_q_Voltage'] = np.interp(raw_track['MCM_Motor_Speed']['Time'][0, 0][0], raw_track['MCM_Phase_BC_Voltage']['Time'][0, 0][0], raw_track['MCM_Phase_BC_Voltage']['Value'][0, 0][0])
data['v_d_Voltage'] = np.interp(raw_track['MCM_Motor_Speed']['Time'][0, 0][0], raw_track['MCM_Phase_AB_Voltage']['Time'][0, 0][0], raw_track['MCM_Phase_AB_Voltage']['Value'][0, 0][0])
data['id_t'] = np.zeros(len(data['time']))
data['iq_t'] = np.zeros(len(data['time']))
data['t_t'] = np.zeros(len(data['time']))
data['w_t'] = np.zeros(len(data['time']))
data['v_max'] = np.zeros(len(data['time']))
data['v_d_t'] = np.zeros(len(data['time']))
data['v_q_t'] = np.zeros(len(data['time']))
data['p_t'] = np.zeros(len(data['time']))
# take only the section between the reliant times
data = data[data['time'] > 30]
data = data[data['time'] < 50]
data = data.reset_index(drop=True)
print(data.shape)

def calcc(w, t, v):
    Id, Iq, T, wbase, v_max = cur_motor.get_qd_currents(w, t, v)
    return Id, Iq, T, wbase * 60 / (2 * np.pi), v_max

# plot the commanded torque vs actual vs calculated theoretical
for i in range(len(data['time'])):
    data['id_t'][i], data['iq_t'][i], data['t_t'][i], data['w_t'][i], data['v_max'][i] = calcc(data['w'][i], data['torque_command'][i], data['dc_voltage'][i])

for i in range(len(data['time'])):
    data['v_d_t'][i], data['v_q_t'][i] = cur_motor.get_voltages(data['w'][i], data['id_t'][i], data['iq_t'][i])
    data['p_t'][i] = np.sqrt(data['v_d_t'][i]**2 + data['v_q_t'][i]**2) * np.sqrt(data['id_t'][i]**2 + data['iq_t'][i]**2)
fig = make_subplots(rows=7, cols=1, shared_xaxes=True)
fig.add_trace(go.Scattergl(x=data['time'], y=data['torque_command'], mode='lines', name='Commanded Torque', line=dict(color='blue', width=2)), row=1, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['torque_feedback'], mode='lines', name='Feedback Torque', line=dict(color='green', width=2)), row=1, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['t_t'], mode='lines', name='Theoretical Torque', line=dict(color='red', width=2)), row=1, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['i_d'], mode='lines', name='i_d', line=dict(color='blue', width=2)), row=2, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['i_q'], mode='lines', name='i_q', line=dict(color='green', width=2)), row=2, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['i_d_commanded'], mode='lines', name='i_d_commanded', line=dict(color='blue', width=2)), row=2, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['i_q_commanded'], mode='lines', name='i_q_commanded', line=dict(color='green', width=2)), row=2, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['id_t'], mode='lines', name='i_d Theoretical', line=dict(color='red', width=2)), row=2, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['iq_t'], mode='lines', name='i_q Theoretical', line=dict(color='orange', width=2)), row=2, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['dc_voltage'], mode='lines', name='DC Voltage', line=dict(color='blue', width=2)), row=3, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['dc_current'], mode='lines', name='DC Current', line=dict(color='green', width=2)), row=3, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['v_d_Voltage'], mode='lines', name='V_d Voltage', line=dict(color='blue', width=2)), row=4, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['v_q_Voltage'], mode='lines', name='V_q Voltage', line=dict(color='green', width=2)), row=4, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['v_d_t'], mode='lines', name='V_d Theoretical', line=dict(color='red', width=2)), row=4, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['v_q_t'], mode='lines', name='V_q Theoretical', line=dict(color='orange', width=2)), row=4, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['v_max'], mode='lines', name='V_max', line=dict(color='green', width=2)), row=4, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['w'], mode='lines', name='Speed', line=dict(color='blue', width=2)), row=5, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['w_t'], mode='lines', name='Theoretical Speed', line=dict(color='red', width=2)), row=5, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['p_t'], mode='lines', name='Theoretical Power', line=dict(color='red', width=2)), row=6, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['dc_voltage']*data['dc_current'], mode='lines', name='Power', line=dict(color='red', width=2)), row=6, col=1)
fig.add_trace(go.Scattergl(x=data['time'], y=data['dc_voltage']*data['dc_current'] - data['p_t'], mode='lines', name='Power Difference', line=dict(color='red', width=2)), row=7, col=1)


fig.update_layout(template="plotly_dark", title_text=f"Torque Command vs Feedback", height=1000, width=1000, xaxis_title="Time", yaxis_title="Torque (Nm)")
fig.show()

In [ ]:

def generate_frames(df, total=0):
    fig = go.Figure()
    fig.update_xaxes(title_text='Id (A)')
    fig.update_yaxes(title_text='Iq (A)')

    # shift the data index to start at 0
    og_ind = df.index[0]
    df.index = df.index - df.index[0]

    current_lims = 600
    i_dmtpa, i_qmtpa = cur_motor.calc_mtpa(np.linspace(0, 2 * cur_motor.i_max, 100))
    fig.add_trace(go.Scatter(x=i_dmtpa, y=i_qmtpa, mode='lines', name='Max Torque Per Amp', line=dict(color='green', width=2)))
    winding_current_limit = cur_motor.i_max
    fig.add_shape(type="circle", xref="x", yref="y", x0=-winding_current_limit, y0=-winding_current_limit, x1=winding_current_limit, y1=winding_current_limit, line_color="red", name='Motor Current Limit')
    i_d_eeprom, i_q_eeprom = create_rectangle(350, 120)
    fig.add_trace(go.Scatter(x=i_d_eeprom, y=i_q_eeprom, mode='lines', name='MCU EEPROM Current Limit', line=dict(color='orange', width=2)))
    mcu_gate_current_lim = 380
    fig.add_shape(type="circle", xref="x", yref="y", x0=-mcu_gate_current_lim, y0=-mcu_gate_current_lim, x1=mcu_gate_current_lim, y1=mcu_gate_current_lim, line_color="purple", name='MCU Gate Current Limit')
    # add labels showing the color of the circle for the current limits
    fig.add_trace(go.Scatter(x=[-current_lims], y=[-current_lims], mode='lines', name='Motor Current Limit', marker=dict(color='red', size=4)))
    fig.add_trace(go.Scatter(x=[-current_lims], y=[-current_lims - 50], mode='lines', name='MCU EEPROM Current Limit', marker=dict(color='purple', size=4)))

    obj_steps = [(0, 0)] * df.shape[0]
    current_index = 4
    for i in range(len(obj_steps)):
        torque = df['torque_command'][i]
        speed = df['w'][i]
        voltage = df['dc_voltage'][i]
        i_d_torque = np.linspace(-current_lims, current_lims, 100)
        i_q_torque = cur_motor.calc_iq(torque, i_d_torque)
        fig.add_trace(go.Scatter(x=i_d_torque, y=i_q_torque, mode='lines', name='Requested Torque Curve', line=dict(color='blue', width=2), visible=False))
        i_d_emf, i_q_emf = cur_motor.calc_emf_limit(speed, voltage, 100)
        fig.add_trace(go.Scatter(x=i_d_emf, y=i_q_emf, mode='lines', name='EMF Circle', line=dict(color='white', width=2), visible=False))
        fig.add_trace(go.Scatter(x=[df['i_d'][i]], y=[df['i_q'][i]], mode='markers', name='Current', marker=dict(color='white', size=4), visible=False))
        fig.add_trace(go.Scatter(x=[df['i_d_commanded'][i]], y=[df['i_q_commanded'][i]], mode='markers', name='Commanded Current', marker=dict(color='red', size=4), visible=False))
        obj_range = (current_index, current_index + 4)
        current_index += 4
        obj_steps[i] = obj_range

    for ob in fig.data[obj_steps[0][0]:obj_steps[0][1]]: ob.visible = True

    steps = []
    for i in range(len(obj_steps)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)}],  # layout attribute
            label=f"{df['time'][i]:.4f} s"
        )
        step["args"][0]["visible"][0:4] = [True] * 4
        step["args"][0]["visible"][obj_steps[i][0]:obj_steps[i][1]] = [True] * (obj_steps[i][1] - obj_steps[i][0])
        steps.append(step)

    fig.update_layout(title_text=f"Iq and Id Current Target", height=1024, width=1024 , template="plotly_dark")
    fig.update_xaxes(range=[-current_lims, current_lims])
    fig.update_yaxes(range=[-current_lims, current_lims])
    # create a annotation for the time
    fig.update_layout(annotations=[dict(text="", x=1.0, y=1.1, xref="paper", yref="paper", showarrow=False),
                                    dict(text="", x=1.0, y=1.075, xref="paper", yref="paper", showarrow=False),
                                    dict(text="", x=1.0, y=1.05, xref="paper", yref="paper", showarrow=False),
                                    dict(text="", x=1.0, y=1.025, xref="paper", yref="paper", showarrow=False),
                                    dict(text="", x=1.0, y=1.0, xref="paper", yref="paper", showarrow=False),
                                    dict(text="", x=1.0, y=0.975, xref="paper", yref="paper", showarrow=False),
                                    dict(text="Time:", x=0.85, y=1.1, xref="paper", yref="paper", showarrow=False),
                                    dict(text="Torque Cmd:", x=0.85, y=1.075, xref="paper", yref="paper", showarrow=False),
                                    dict(text="Speed:", x=0.85, y=1.05, xref="paper", yref="paper", showarrow=False),
                                    dict(text="Voltage:", x=0.85, y=1.025, xref="paper", yref="paper", showarrow=False),
                                    dict(text="Current:", x=0.85, y=1.0, xref="paper", yref="paper", showarrow=False),
                                    dict(text="Torque Feedback:", x=0.85, y=0.975, xref="paper", yref="paper", showarrow=False)])


    frames = []
    start_time = time.time()
    for s, fr in enumerate(steps):
        # add label for the current time
        fig.layout.annotations[0].text = f"{df['time'][s]:.3f} s"
        fig.layout.annotations[1].text = f"{df['torque_command'][s]:.1f}  Nm"
        fig.layout.annotations[2].text = f"{df['w'][s]:.1f} RPM"
        fig.layout.annotations[3].text = f"{df['dc_voltage'][s]:.1f}     V"
        fig.layout.annotations[4].text = f"{df['dc_current'][s]:.1f}     A"
        fig.layout.annotations[5].text = f"{df['torque_feedback'][s]:.1f}  Nm"
        lower, upper = max(obj_steps[s][0]-4, 2), obj_steps[s][1]
        for i in range(lower, upper):
            fig.data[i].visible = fr["args"][0]["visible"][i]
        # generate image of current state
        layout_update_time = time.time()
        img = fig.to_image(format="png")
        img_time = time.time()
        img_bytes = io.BytesIO(img)
        bytes_time = time.time()
        pil_img = PIL.Image.open(img_bytes)
        frames.append(pil_img)
        # frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
        # print(f"Frame {s} of {len(steps)}: {time.time() - start_time:.3f} s\t{layout_update_time - start_time:.3f} s\t{img_time - layout_update_time:.3f} s\t{bytes_time - img_time:.3f} s\t{time.time() - bytes_time:.3f} s\t{time.time() - start_time:.3f} s")
        print(f"Frame {s + og_ind} of {total}: {time.time() - start_time:.3f} s")
        start_time = time.time()
    return frames
    

animation_frames = []
for i in range(0, data.shape[0], 10):
    animation_frames.extend(generate_frames(data.iloc[i:i+10], total=data.shape[0]))
# create animated GIF
animation_frames[0].save(
        "test.gif",
        save_all=True,
        append_images=animation_frames[1:],
        optimize=True,
        duration=50,
        loop=0,
    )